In [ ]:
"""
Created on Thu Apr 28 11:25 2022

This script is to compare param and ref melt (scatter and timeseries)

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from basal_melt_param.constants import *
import seaborn as sns
from matplotlib import cm
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import glob

READ IN DATA

In [ ]:
#home_path = '/Users/claraburgard/bettik_clara'
home_path='/bettik/burgardc/'
plot_path = home_path+'/PLOTS/NN_plots/1D_eval/'

In [ ]:
nemo_run0 = 'OPM026' #['OPM006','OPM016','OPM018','OPM021','OPM026','OPM027','OPM031-1','OPM031-2']

if nemo_run0 in ['OPM031-1','OPM031-2']:
    nemo_run = 'OPM031'
else:
    nemo_run = nemo_run0
    

In [ ]:
inputpath_mask = home_path+'DATA/BASAL_MELT_PARAM/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)


In [ ]:
### READ IN THE REFERENCE
outputpath_melt = home_path+'DATA/BASAL_MELT_PARAM/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
NEMO_melt_rates_1D = xr.open_dataset(outputpath_melt+'melt_rates_1D_NEMO.nc')
if nemo_run0 == 'OPM031-1':
    ref_Gt = NEMO_melt_rates_1D['melt_Gt_per_y_tot'].isel(time=range(30))
elif nemo_run0 == 'OPM031-2':
    ref_Gt = NEMO_melt_rates_1D['melt_Gt_per_y_tot'].isel(time=range(30,70))
else:
    ref_Gt = NEMO_melt_rates_1D['melt_Gt_per_y_tot']
NEMO_box1_myr = xr.open_dataset(outputpath_melt+'melt_rates_box1_NEMO.nc')
ref_box1 = NEMO_box1_myr['mean_melt_box1_myr']


In [ ]:
outputpath_melt_nn = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/'
timetag_list = ['20220427-1059','20220427-1051']

In [ ]:
diff_Gt_list = []
diff_box1_list = []
diff_Gt_anom_list = []
diff_box1_anom_list = []

ref_Gt_list = []
ref_box1_list = []

param_Gt_list = []
param_box1_list = []

file_isf_list = []



for tt,timetag in enumerate(timetag_list):
    
    new_path_output = outputpath_melt_nn+timetag+'/'
    
    ### READ IN THE PARAM FILES
    
    # Param files

    ds_melt_param = xr.open_dataset(new_path_output+'eval_metrics_'+nemo_run0+'.nc')
    
    param_Gt = ds_melt_param['melt_1D_Gt_per_y']
    diff_Gt = ds_melt_param['melt_1D_Gt_per_y'] - ref_Gt
    diff_Gt_anom = (ds_melt_param['melt_1D_Gt_per_y'] - ds_melt_param['melt_1D_Gt_per_y'].mean('time'))  - (ref_Gt - ref_Gt.mean('time'))
    #diff_Gt = diff_Gt.assign_coords({'time': np.arange(1,len(diff_Gt.time)+1)+n*50})
    diff_Gt_list.append(diff_Gt)
    diff_Gt_anom_list.append(diff_Gt_anom)
    param_Gt_list.append(param_Gt)
    
    param_box1 = ds_melt_param['melt_1D_mean_myr_box1'].mean('time')
    diff_box1 = ds_melt_param['melt_1D_mean_myr_box1'].mean('time') - ref_box1.mean('time')
    diff_box1_list.append(diff_box1)  
    param_box1_list.append(param_box1)
    
    
diff_Gt_all = xr.concat(diff_Gt_list, dim='nn_model')
diff_Gt_all = diff_Gt_all.assign_coords(nn_model=timetag_list)
diff_Gt_anom_all = xr.concat(diff_Gt_anom_list, dim='nn_model')
diff_Gt_anom_all = diff_Gt_anom_all.assign_coords(nn_model=timetag_list)
diff_box1_all = xr.concat(diff_box1_list, dim='nn_model') 
diff_box1_all = diff_box1_all.assign_coords(nn_model=timetag_list)

param_box1_all = xr.concat(param_box1_list, dim='nn_model')
param_box1_all = param_box1_all.assign_coords(nn_model=timetag_list)

param_Gt_all = xr.concat(param_Gt_list, dim='nn_model')
param_Gt_all = param_Gt_all.assign_coords(nn_model=timetag_list)



In [ ]:
param_melt_Gt_yr = param_Gt_all
target_melt_Gt_yr = ref_Gt

In [ ]:
region_list = []
for kisf in file_isf.Nisf:
    if file_isf['isf_name'].sel(Nisf=kisf).values in ['Ross','Nickerson','Sulzberger', 'Cook']:
        region_list.append('East and West Ross')
    elif file_isf['isf_name'].sel(Nisf=kisf).values in ['Filchner','Ronne']:
        region_list.append('Weddell')        
    elif file_isf['isf_name'].sel(Nisf=kisf).values in ['Ekstrom','Nivl','Prince Harald','Riiser-Larsen','Fimbul','Roi Baudouin','Lazarev','Stancomb Brunt','Jelbart','Borchgrevink']:
        region_list.append('Dronning Maud Land')
    elif file_isf['isf_name'].sel(Nisf=kisf).values in ['Getz','Thwaites','Crosson','Dotson','Cosgrove','Pine Island']:
        region_list.append('Amundsen')
    elif file_isf['isf_name'].sel(Nisf=kisf).values in ['Venable','George VI','Abbot','Stange','Larsen C','Bach','Larsen D','Wilkins']:
        region_list.append('Peninsula and Bellinghausen')
    elif file_isf['isf_name'].sel(Nisf=kisf).values in ['Amery','Moscow Univ.','Tracy Tremenchus','Totten','West','Shackleton']:
        region_list.append('East Antarctica')
    else:
        print('Argh, help me, '+file_isf['isf_name'].sel(Nisf=kisf).values+' has no region assigned!')
# 6 regions
# regions = ['East and West Ross','Weddell','Dronning Maud Land','Amundsen','Peninsula and Bellinghausen','East Antarctica']
file_isf['region'] = xr.DataArray(data=region_list,dims='Nisf')

In [ ]:
def plot_melt_timeseries_all_isf(param_melt_tuned, target_Gt_yr, file_isf, nemo_run):
    
    regions = ['Weddell','Peninsula and Bellinghausen','Amundsen','East and West Ross','East Antarctica','Dronning Maud Land']
    colors = ['deepskyblue','brown','red','orange','limegreen','seagreen']
    symbol = ['o','v','>','p','*','s','<','^','X','d']
    
    f = plt.figure()
    f.set_size_inches(8.25*1.5, 8.25*1.5)

    ax={}

    i = 0
    nn=0
    ii=0
    marker_symbol = np.zeros((len(regions))).astype(int)
    for rr,reg in enumerate(regions):
        k = 0
        marker_color = colors[rr]
        subset_isf = file_isf.Nisf.where(file_isf['region']==reg,drop=True)
        #print(subset_isf)
        for kisf in tqdm(subset_isf.Nisf):
            
            if i > 0:
                ax[i] = f.add_subplot(6,6,i+1, sharex=ax[0])
            else:
                ax[i] = f.add_subplot(6,6,i+1)          

            target = target_Gt_yr.sel(Nisf=kisf)
            target.plot(ax=ax[i], c='blue')
            
            for tt,timetag in enumerate(param_melt_tuned.nn_model):
                
                if timetag in ['20220427-1021','20220427-1042','20220427-1051']:
                    ls = '--'
                elif timetag in ['20220427-1052','20220427-1059']:
                    ls = '-'
                parameterised = param_melt_tuned.sel(Nisf=kisf).sel(nn_model=timetag)
                parameterised.plot(ax=ax[i], c=colors[tt],linestyle=ls)


            ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
            f.suptitle(nemo_run)

            i = i+1
            k = k+1


    f.tight_layout()
    sns.despine()

    return f

In [ ]:
nemo_run

In [ ]:
f = plot_melt_timeseries_all_isf(param_melt_Gt_yr, target_melt_Gt_yr, file_isf, nemo_run0)
f.savefig(plot_path+'timeseries_melt_'+nemo_run0+'_comparelargemodels.png')